## 1. 重点复习

自连接语法   表  表1  inner join 表 表 2 on 表1.字段  条件 表 2.字段

子查询<标量、行、列>

​	select * from students where height > (select avg(height) from students);

​	select * from students where (age,height) = (select max(age),max(height) from students)

​	select name from gongfu where id in (select gongfuid from students);



数据库编程

​	连接数据库  连接对象= pymysql.connect(host,port,user,password,database,charset)

​	获取游标  游标对象 = 连接对象.cursor()

​	执行 SQL SQL影响函数= 游标对象.execute(sql,[sql 所需参数])

​	获取结果/提交数据

​	关闭资源  先游标  再连接



事务

数据库设计

​	ER		

>   letcode SQL

## 2. 数据库

### 2.1 扩展SQL

索引SQL 语句扩展

	如果删除了 8910 号商品导致平板电脑分类没了
	如果删除了 20 号商品导致 ibm 品牌没了              删除异常
	为解决问题  将商品信息  和 分类信息以及 品牌信息单独拆分独立的表格
1.  要将商品表中的分类信息单独放到一个分类表中

    ```sql
    -- 创建商品分类表
    create table goods_cates(
        id int not null primary key auto_increment, 
        name varchar(50) not null
    );
    ```

    insert into 表（字段）values (),()

    将查询结果直接插入表中

    mysql> insert into goods_cates(name) select cate_name from goods group by cate_name;

2.  要将商品表中的品牌信息单独放到一个品牌表中

    select brand_name from goods group by brand_name;

    ```sql
    -- 创建表的同时就插入数据  查询出的字段和表字段中同名
    create table goods_brands (      
    id int unsigned primary key auto_increment,      	name varchar(40) not null) 
    select brand_name name from goods group by brand_name;
    ```

3.  连表更新

    select * from `表1 inner join 表2 on 表1.字段 条件 表2.字段`;

     `表1 inner join 表2 on 表1.字段 条件 表2.字段`看作一个虚表

    update 表 set 字段=值 where 条件;

    应用场景: 使用一个表的数据去更新另一个表对应的数据

    ![image-20190830101636965](day11笔记.assets/image-20190830101636965.png)

    ```sql
    -- 使用goods_cates 的 id 去更新  goods 表中 cate_name
    update goods inner join goods_cates on goods.cate_name = goods_cates.name
    	set goods.cate_name = goods_cates.id;
    
    -- 使用goods_brands 的 id 去更新  goods 表中 brand_name
    update goods inner join goods_brands on goods.brand_name = goods_brands.name 
    set goods.brand_name = goods_brands.id;
    ```

    形式:

    >   修改表的字段名和 类型
    >
    >   alter table goods change cate_name cate_id int unsigned not null,
    >    change brand_name brand_id int unsigned not null;
    >
    >   多表联合 、连接查询 
    >
    >   select * from goods 
    >
    >   ​	inner join goods_cates on goods.cate_id = goods_cates.id
    >
    >   ​	inner join goods_brands on goods.brand_id = goods_brands.id
    >
    >   ​	innert join 表 on 表.字段=表.字段
    
 ### 2.2 京东案例

将所有的笔记本的分类改为超级本

​	查询笔记本的分类 ID   select id from goods_cates where name = '笔记本';

​	查询超级本的分类 ID   select id from goods_cates where name = '超级本';

​	update goods set cate_id = (select id from goods_cates where name = '超级本')

​		where cate_id = (select id from goods_cates where name = '笔记本');



### 2.3 索引

​	作用:   

​		索引是有序的

​		提高数据的**查询**效率  因为索引能够快速定位到数据所在的位置

​	使用: 

​		查看   show index from 表名\G; \G 能够将每个字段用一行显示

​			show index from goods_cates;

​		添加   alter table 表名 add index 索引名(索引字段名);

​			alter table goods_cate add index xxxx(name);

​		删除   alter table goods_cates drop index 索引名;

​			alter table goods_cate drop index  xxxx;

​	技巧:

​		**联合索引最左原则: 从最左边连续取一个或多个**		

​		更新数据时 效率反而会更低

​		索引虽好 不要贪杯，在大量查询的字段上创建索引。

In [ ]:
import pymysql


class JD(object):
    def __init__(self):
        """初始化属性"""
        self.conn = pymysql.connect(host='localhost',port=3306,user='root',password='mysql',db='jing_dong',charset='utf8')
    def __del__(self):
        self.conn.close()  # 当当前对象退出时　　就会自动调用
    def show_menu(self):
        print("－"*30)
        print('1. 查询所有商品信息')
        print("2. 查询所有包含商品的分类")
        print("3. 添加新商品分类")
        print("4. 将所有商品价格加1000")
        print("5. 将所有笔记本的分类改为超级本")
        print("6. 根据id查询商品信息")
        print("7. 根据id查询商品信息-防SQL注入")
        print("8. 退出系统")

    def get_all_info(self):
        print("开始查询所有商品信息")
        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        sql = "select * from goods;"
        cur.execute(sql)
        # 3 打印结果
        for line in cur.fetchall():
            print(line)

        # 4 关闭游标
        cur.close()
    def get_all_cate(self):
        print("－－－－－－开始查询所有商品分类－－－－－－－－－－－－－")
        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        # sql = "select distinct cate_name from goods;"
        sql = "select name from goods_cates;"
        cur.execute(sql)
        # 3 打印结果
        for line in cur.fetchall():
            print(line)

        # 4 关闭游标
        cur.close()

    def add_new_cate(self):
        print("－－－－－－开始新增分类信息－－－－－－－－－－－－－")
        cate_name = input("请输入需要添加的分类:")

        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        sql = "insert into goods_cates(name) VALUES (%s)"  # ('蓝牙音箱'),('智慧屏')
        cur.execute(sql,[cate_name])
        # 3 提交修改
        self.conn.commit()

        # 4 关闭游标
        cur.close()
    def update_price(self):
        print("－－－－－－开始修改商品价格－－－－－－－－－－－－－")
        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        sql = "update goods set price=price +1000 ;"
        cur.execute(sql)
        # 3 提交修改
        self.conn.commit()

        # 4 关闭游标
        cur.close()

    def update_cate(self):
        print("－－－－－－开始修改商品分类信息－－－－－－－－－－－－－")
        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句　ＯＲＭ用面向对象的方式操作数据库
        sql = """update goods set cate_id = (select id from goods_cates where name = '超级本')
                where cate_id = (select id from goods_cates where name = '笔记本');"""

        cur.execute(sql)
        # 3 提交修改
        self.conn.commit()

        # 4 关闭游标
        cur.close()

    def get_info_by_id(self):
        print("－－－－－－根据ＩＤ开始查询商品信息－－－－－－－－－－－－－")
        gid = input("请输入你需要查询的ID:")

        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        sql = "select * from goods WHERE id = %s" % gid
        cur.execute(sql)
        # 3 打印结果
        for line in cur.fetchall():
            print(line)

        # 4 关闭游标
        cur.close()
    def get_info_by_id_safe(self):
        print("－－－－－－根据ＩＤ开始查询商品信息 防止注入－－－－－－－－－－－－－")
        gid = input("请输入你需要查询的ID:")

        # 1 获取游标
        cur = self.conn.cursor()

        # 2 执行ＳＱＬ语句
        sql = "select * from goods WHERE id = %s"
        cur.execute(sql, [gid])
        # 3 打印结果
        for line in cur.fetchall():
            print(line)

        # 4 关闭游标
        cur.close()

    def start(self):
        """负责和用户进行交互　需要用进行输入　执行对应功能　打印出最后结果"""
        while True:
            # 打印菜单
            self.show_menu()
            op = input("请输入你的选项:")  # 接收用户输入
            if op == '1':
                self.get_all_info()
            elif op == '2':
                self.get_all_cate()
            elif op == '3':
                self.add_new_cate()
            elif op == '4':
                self.update_price()
            elif op == '5':
                self.update_cate()
            elif op == '6':
                self.get_info_by_id()
            elif op == '7':
                self.get_info_by_id_safe()
            else:
                print("客官欢迎下次再来")
                break


if __name__ == '__main__':
    jd = JD()
    jd.start()